In [1]:
import json
import os
from sklearn.model_selection import train_test_split
import csv
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import cv2

Adjust the below according the directories of new folder location and current bagls location

In [24]:
bagls = r"D:\FYP Datasets\BAGLS" #to change to the path of the BAGLS dataset
output = r"D:\FYP Datasets\fyp" #to change to the path of the output folder

In [ ]:
bagls_test_t = os.path.join(bagls, "test", "test")
# bagls_test_a = os.path.join(bagls, "test", "annotations") #if bbox is done
bagls_train_t = os.path.join(bagls, "training", "training")
# bagls_train_a = os.path.join(bagls, "training", "annotations") #if bbox is done

o_images = os.path.join(output, "raw", "images")
o_labels = os.path.join(output, "raw", "labels")

#create the directory for the output folder
os.makedirs(o_images, exist_ok=True)
os.makedirs(o_labels, exist_ok=True)

#for training 
train_img = os.path.join(output, "training", "train", "images")
train_label = os.path.join(output, "training", "train", "labels")
val_img = os.path.join(output, "training", "val", "images")
val_label = os.path.join(output, "training", "val", "labels")
test_img = os.path.join(output, "training", "test", "images")
test_label = os.path.join(output, "training", "test", "labels")

os.makedirs(train_img, exist_ok=True)
os.makedirs(train_label, exist_ok=True)
os.makedirs(val_img, exist_ok=True)
os.makedirs(val_label, exist_ok=True)
os.makedirs(test_img, exist_ok=True)
os.makedirs(test_label, exist_ok=True)

## Copy over data files from bagls to output folder ##




Done with training data first followed by test data.
For test data, everything will be renamed starting from the last index of the training data.

In [ ]:
#copying train images and annotations
for filename in os.listdir(bagls_train_t):
    shutil.copy2(os.path.join(bagls_train_t, filename), o_images)

#if bbox is done
# for filename in os.listdir(bagls_train_a):
#     shutil.copy2(os.path.join(bagls_train_a, filename), o_labels)

In [ ]:
lastnum = 0
checker = 0
for file in os.listdir(o_images):
    if file.endswith(".png"):
        lastnum += 1

#if bbox is done    
# for file in os.listdir(o_labels):
#     if file.endswith(".txt"):
#         checker += 1

In [ ]:
#copying test images and annotations and rename starting from the last number
counter1,counter2,counter3 = lastnum, lastnum, lastnum
for filename in os.listdir(bagls_test_t):
    if filename.endswith("_seg.png"):
        shutil.copy2(os.path.join(bagls_test_t, filename), os.path.join(o_images, str(counter1)+"_seg.png"))
        counter1 += 1
    elif filename.endswith(".meta"):
        shutil.copy2(os.path.join(bagls_test_t, filename), os.path.join(o_images, str(counter2)+".meta"))
        counter2 += 1
    elif filename.endswith(".png") and not filename.endswith("_seg.png"):
        shutil.copy2(os.path.join(bagls_test_t, filename), os.path.join(o_images, str(counter3)+".png"))
        counter3 += 1
        
#only if bbox is done
# for filename in os.listdir(bagls_test_a):
#     f = int(filename.split(".")[0])
#     shutil.copy2(os.path.join(bagls_test_a, filename), os.path.join(o_labels, str(f+lastnum)+".txt"))

In [8]:
#checker for the number of images and labels
print("Number of images: ", len(os.listdir(o_images)))

Number of images:  59250


## Finds bounding boxes all iamges and creates an label file in the images folder #

only done for bagls dataset

In [ ]:
def detect_bounding_boxes(img):
    x_min, y_min, x_max, y_max = float('inf'), float('inf'), 0, 0
    cntrs, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in cntrs:
        x, y, w, h = cv2.boundingRect(c)
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x + w)
        y_max = max(y_max, y + h)
    #convert to yolo
    x_center = (x_min + x_max) / 2 / img.shape[1]
    y_center = (y_min + y_max) / 2 / img.shape[0]
    width = (x_max - x_min) / img.shape[1]
    height = (y_max - y_min) / img.shape[0]
    return x_center, y_center, width, height



In [ ]:
for file in os.listdir(o_images):
    if file.endswith("_seg.png"):
        img = cv2.imread(os.path.join(o_images, file), cv2.IMREAD_GRAYSCALE)
        x, y, w, h = detect_bounding_boxes(img)
        if x == float('inf') or y == float('inf'):
            continue
        #write in a new text file and save in another folder
        with open(os.path.join(o_labels, file.replace("_seg.png", ".txt")), "w") as f: #replace _seg.png with .txt to follow 
            f.write(f"3 {x} {y} {w} {h}")

In [9]:
#checker
print("Expected labels from BAGLS is 55,286")
print("Number of labels: ", len(os.listdir(o_labels))) 

Expected labels from BAGLS is 55,286
Number of labels:  59250


## Creating metadata file for the dataset

*needs to be done before adding negative samples*

In [ ]:
meta_file = os.path.join(o_images, "0.meta")
with open(meta_file, 'r') as f:
    data = json.load(f)
    headers = data.keys()
    headers = list(headers)
    headers.append('glottis status')
    headers.append('Type')
    headers.insert(0, 'Filename')
    with open(output + "/metadata.csv" , 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)

In [ ]:
#append data into csvfile and parse metadata
counter = 0
with open(os.path.join(output, 'metadata.csv'), 'a', newline='') as f:
    writer = csv.writer(f)
    for filename in os.listdir(o_images):
        if filename.endswith(".meta"):
            name = filename.split(".")[0]
            with open(os.path.join(o_images, filename), 'r') as file:
                data = json.load(file)
                relevant_data = [name] # add filename to the list
                
                for key in data.keys():
                    relevant_data.append(data[key]) #add metadata to list
                    
                    label_exist = False
                    for label in os.listdir(o_labels):
                        if label.split(".")[0] == name:
                            label_exist = True #add glottis status to list 
                            break
                    if label_exist:
                        relevant_data.append("Open")
                    else:
                        relevant_data.append("Closed")

                if counter < lastnum: #add data type to list
                    relevant_data.append("train")
                else:
                    relevant_data.append("test")
                counter += 1
                writer.writerow(relevant_data)


## Adding Negative Samples Annotation File

firstly remove all .meta files and _seg.png files


In [ ]:
#remove all files except images 
a = 0
for file in os.listdir(o_images):
    if file.endswith(".meta"):
        os.remove(os.path.join(o_images, file))
        a += 1
    if file.endswith("_seg.png"):
        os.remove(os.path.join(o_images, file))
        a += 1

if a == 118500: # total remaining files should be 55950
    print("Files removed successfully")

add empty .txt files for those missing labels

*For fyp dataset*

In [5]:
#check which files are missing
c = 0
for file in os.listdir(o_images):
    name = file.split(".")[0]
    if not os.path.exists(os.path.join(o_labels, name + ".txt")):
        c += 1
        # print(name) 
        with open(os.path.join(o_labels, name + ".txt"), "w") as f:
            pass

print("Initial Number of missing annotations: ", 3964)
print("Number of missing annotations: ", c)
print("Total number of images: ", len(os.listdir(o_images)))
print("Total number of annotations: ", len(os.listdir(o_labels)))

Initial Number of missing annotations:  3964
Number of missing annotations:  0
Total number of images:  59250
Total number of annotations:  59250


## Splitting dataset to train, val and test

the splitting will be done on the paths of the images and labels

In [28]:
#create a list of all images values

images_path = sorted(os.listdir(o_images))
labels_path = sorted(os.listdir(o_labels))

#split the data into training and validation 
X_train, X_temp, y_train, y_temp = train_test_split(images_path, labels_path, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.1, random_state=42)

#copy the images and labels to the respective folders
for image in X_train:
    shutil.copy2(os.path.join(o_images, image), train_img)
for label in y_train:
    shutil.copy2(os.path.join(o_labels, label), train_label)
for image in X_val:
    shutil.copy2(os.path.join(o_images, image), val_img)
for label in y_val:
    shutil.copy2(os.path.join(o_labels, label), val_label)
for image in X_test:
    shutil.copy2(os.path.join(o_images, image), test_img)
for label in y_test:
    shutil.copy2(os.path.join(o_labels, label), test_label)
    
print("Number of training images: ", len(os.listdir(train_img)))
print("Number of training labels: ", len(os.listdir(train_label)))
print("Number of validation images: ", len(os.listdir(val_img)))
print("Number of validation labels: ", len(os.listdir(val_label)))
print("Number of test images: ", len(os.listdir(test_img)))
print("Number of test labels: ", len(os.listdir(test_label)))




Number of training images:  47400
Number of training labels:  47400
Number of validation images:  10665
Number of validation labels:  10665
Number of test images:  1185
Number of test labels:  1185
